**前向分步算法**  

学习模型是加法模型，并且学习的过程是串行；从开始到最后，每一步只学习一个基函数及其系数，并对损失函数最优化；从而将整体加法模型的最优化，简化为每个基模型的最优化。  

**AdaBoost与前向分布算法**  
上一节的AdaBoost的算法是串行训练多个模型，然后最后用加权和的方法进行集成。  
对这个算法进行拆分：  
* 串行训练方法 类似于 前向分步算法  
* 加权和  类似于  加法模型
* 损失函数是指数函数（可以证明，奈何还在补数学中，不会~）

因此，AdaBoost算法是前向分步算法的一个特例。

**提升树**  

首先**提升方法**实际采用加法模型（即基函数的线性组合）与前向分步算法。因此，以决策树为基函数的提升方法称为**提升树**。在上一节我们实现的AdaBoost算法（以决策树为基函数）就是一种**分类问题**的提升树。如果需要将提升树应用于**回归模型**，只需要将损失函数改为**误差损失函数**即可。

**具体算法步骤如下：**  
数据集：$T=\left\{\left(x_{1}, y_{1}\right),\left(x_{2}, y_{2}\right), \cdots,\left(x_{N}, y_{N}\right)\right\}, x_{i} \in \mathcal{X} \subseteq \mathbf{R}^{n}, y_{i} \in \mathcal{Y} \subseteq \mathbf{R}$  
基模型为树模型：$T(x;\theta)$  
误差损失函数(平方误差)：$L(y,f(x)) = (y-f(x))^2$  
学习算法为前向分步算法，步骤如下：  
* $f_0(x)=0$
* $f_m(x)=f_{m-1}+T(x;\theta_m)$
* $f_M(x) = \sum_{m=1}^M T(x;\theta_m)$  

在前向分步算法的第m步时，需要求解：  
$\hat \theta_m = arg min \sum_{i=1}^{N} L(y_i, f_{m-1}(x_i) + T(x_i;\theta_m))$  
求得$\theta_m$，即m棵树的参数。

当采用平方误差损失函数时，损失为：
$
L(y, f(x)) = (y-f(x))^2
\longrightarrow
L(y, f_{m-1} + T(x;\theta_m)) = (y-f_{m-1}(x)-T(x;\theta_m))^2  
$  
  
令$r=y-f_{m-1}(x)$,则：   
$L(y, f_{m-1} + T(x;\theta_m)) = (r-T(x;\theta_m))^2  $

r是前一模型训练后的差值，因此对于回归问题的提升树来讲，需要拟合的就是前一模型的差值。

**GBDT**   

当损失函数是平方损失和函数时，每一步的优化相对容易；但是对于一般的损失函数而言，往往每一步的优化并不容易。因此，需要提出更好的优化方法。

针对上面的问题，Freidman提出了**梯度提升算法**(gradient boosting)：  
这是**利用最速下降法**的近似方法，利用损失函数的**负梯度**在当前模型的值  

$-\left[\frac{\partial L\left(y, f\left(x_{i}\right)\right)}{\partial f\left(x_{i}\right)}\right]_{f(x) = f_{m-1}(x)}$

作为回归问题提升树算法中的**残差**的**近似值**，拟合回归树。**与其说负梯度作为残差的近似值，不如说残差是负梯度的一种特例。**  

**具体算法步骤如下：**  
数据集：$T=\left\{\left(x_{1}, y_{1}\right),\left(x_{2}, y_{2}\right), \cdots,\left(x_{N}, y_{N}\right)\right\}, x_{i} \in \mathcal{X} \subseteq \mathbf{R}^{n}, y_{i} \in \mathcal{Y} \subseteq \mathbf{R}$  
基模型为回归树模型：$T(x;\theta)$  
损失函数：$L(y,f(x))$  
学习算法为前向分步算法，步骤如下：  


**1, 初始化弱学习器**  
$f_{0}(x)=\arg \min _{c} \sum_{i=1}^{N} L\left(y_{i}, c\right)$ 

损失函数为平方差损失，是凸函数，因此，直接求导=0，可以得到c.

$\sum_{i=1}^N \frac{\partial L(y_i, c)}{\partial c} = \sum_{i=1}^N \frac{\partial (\frac{1}{2}(y_i - c)^2)}{\partial c}=\sum_{i=1}^N c-y_i$ 

**2, 对迭代轮数m=1,2,……M:**  

**2.1 对每个样本，计算负梯度，即残差**  
$r_{im}= -[\frac{\partial L(y_i, f(x_i))}{\partial f(x_i)}]_{f(x)=f_{m-1} (x)}$

**2.2 将上步得到的残差作为样本新的真实值，继续训练新的弱学习器**  
$f_{m}(x)=f_{m-1}(x)+\sum_{j=1}^{J} c_{m j} I\left(x \in R_{m j}\right)$

**3 迭代完m次后，得到最终模型**  
$\hat{f}(x)=f_{M}(x)=\sum_{m=1}^{M} \sum_{j=1}^{J} c_{m j} I\left(x \in R_{m j}\right)$


**sklearn代码实践**  

In [6]:
# 导入库
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_friedman1
from sklearn.ensemble import GradientBoostingRegressor

In [7]:
# 生成测试数据
X, y = make_friedman1(n_samples=1200, random_state=0, noise=1.0)
X_train, X_test = X[:200], X[200:]
y_train, y_test = y[:200], y[200:]

**GradientBoostRegressor参数：**  
* loss:损失，{‘deviance’, ‘exponential’}, default=’deviance’ 
* learning_rate: 学习率，
* n_estimators: 要执行的迭代次数
* subsample：用于拟合各个基学习器的样本比例。如果小于1.0，将使得随机梯度增强。subsample与参数n_estimators有关联，选择subsample<1.0会导致方差减少和偏差增加
* criterion：{friedman_mse，mse，mae}，默认为friedman_mse：mse是均方误差，mae是平均绝对误差。默认值friedman_mse通常是最好的，因为在大多情况下可以提供更好的近似值
* min_samples_split：默认为2，拆分内部节点所需的最少样本数
* min_samples_leaf：默认为1，在叶节点处需要的最小样本数
* min_weight_fraction_leaf：默认为0.0，在所有叶节点处（所有输入样本）的权重总和中的最小加权数。如果未提供sample_weight，则样本的权重相等
* max_depth：默认为3，各个回归模型的最大深度。最大深度限制了树中节点的数量。调整此参数以获得最佳性能；最佳值取决于输入变量
* min_impurity_decrease：如果节点拆分会导致不纯度大于或等于该值，则该节点将被拆分。
* min_impurity_split：提前停止树生长的阈值。如果节点的不纯度高于该值，则该节点将拆分
* max_features {auto, sqrt, log2}，int或float：寻找最佳切分点时要考虑的特征个数：
如果是int，则表示节点切分的特征个数
如果是float，max_features则为小数，根据公式int(max_features * n_features)确定节点切分的特征个数
如果是auto，则max_features=n_features
如果是sqrt，则max_features=sqrt(n_features)
如果为log2，则为max_features=log2(n_features)
如果没有，则max_features=n_features


In [8]:
# 训练
est = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,
    max_depth=1, random_state=0, loss='ls').fit(X_train, y_train)
mean_squared_error(y_test, est.predict(X_test))

5.009154859960321